In [ ]:
from dotenv import load_dotenv, find_dotenv
import urllib.request
import json
import os
import base64
import io
import torch
from PIL import Image

In [32]:
load_dotenv(find_dotenv(), override=True)

True

In [33]:
input_path = "test_input.png"

In [34]:
def preprocess_and_encode_image(image_path):
    """
    Function to load an image, resize if needed, and convert it to base64
    """
    # Open and resize the image
    image = Image.open(image_path)
    image = image.resize((224, 224))  # Example size; check model requirements

    # Convert image to bytes
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    image_bytes = buffered.getvalue()

    # Encode image bytes to base64 string
    image_base64 = base64.b64encode(image_bytes).decode('utf-8')
    return image_base64

#### Virchow

In [35]:
image_base64 = preprocess_and_encode_image(input_path)

data = {
  "input_data": {
    "columns": [
      "image"
    ],
    "index":[0],
    "data": [
      [image_base64]
   ]
 }
}

body = str.encode(json.dumps(data))

url = f'{os.environ.get("VIRCHOW_ENDPOINT")}score'
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = os.environ.get("VIRCHOW_API_KEY")
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = json.loads(response.read().decode('utf-8'))
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

[{'output': [0.40334799885749817, 1.005133867263794, -0.347575306892395, 0.12758059799671173, -1.4507988691329956, -0.027202608063817024, -0.5371573567390442, -0.8064748048782349, 0.32309195399284363, 1.7516043186187744, -0.34083542227745056, 0.9959002733230591, -0.7636595964431763, -0.07247839868068695, 0.17321689426898956, -0.25759586691856384, -0.024884173646569252, 0.8068572282791138, 2.0506513118743896, 0.22460512816905975, -0.696963369846344, -1.8800992965698242, -1.462310552597046, 0.7196168899536133, -0.4351750910282135, -0.21607893705368042, 1.4448620080947876, -0.08207744359970093, 0.45801737904548645, -0.3114601671695709, 0.741625189781189, -2.0171895027160645, -0.004515796434134245, -0.6043871641159058, -0.9240905046463013, -0.35990676283836365, -0.10178851336240768, 1.6206897497177124, 0.4517449736595154, -0.7241015434265137, -1.4905818700790405, 0.9599609375, 1.83512282371521, 0.5981047749519348, 1.1355020999908447, -0.508818507194519, 1.4267603158950806, -0.9007670283317

#### Prism

In [ ]:
embeddings_tensor = torch.tensor(result[0]['output'], dtype=torch.float32)

data = {
  "input_data": {
    "columns": [
      "embeddings",
    ],
    "index":[0],
    "data": [embeddings_tensor]
  }
}

body = str.encode(json.dumps(data))

url = f'{os.environ.get("PRISM_ENDPOINT")}score'
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = os.environ.get("PRISM_API_KEY")
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


The request failed with status code: 424
server: azureml-frontdoor
date: Thu, 14 Nov 2024 21:26:54 GMT
content-type: application/json
content-length: 92
x-ms-run-function-failed: True
x-ms-server-version: azmlinfsrv/1.3.2
x-ms-request-id: d0681f03-39c6-4799-80c3-37287698b414
x-request-id: d0681f03-39c6-4799-80c3-37287698b414
ms-azureml-model-error-reason: model_error
ms-azureml-model-error-statuscode: 500
azureml-model-deployment: prism
azureml-model-session: prism
connection: close


{"message": "An unexpected error occurred in scoring script. Check the logs for more info."}
